In [1]:
# default_exp retrieve

In [2]:
import config
import requests
import base64
import json
import pandas as pd

# retrieval

> Pull all tracks and their features for a user's public playlists

In [3]:
#hide
from nbdev.showdoc import *

In [4]:
with open('spot_tokens.json') as file:
    tokens = json.load(file)

In [5]:
access_token = tokens['access_token']
refresh_token = tokens['refresh_token']

In [6]:
pref = {
# 'panthera_onca': "29U1jhpnjkwziC4pcsvpml",
# 'ten': "6K5607lu5oKcfV5hVRhjtQ",
# 'deep_progressive': "3DSMq4VymDCJClXlcodgW1",
# 'panthers_are_jaguars': "2aipXVr9Sl4YlXHK5BBzWN",
# 'humanism': '5iENhVW0Y6WmptwabKSHla',
# 'all day i dream of releases': '6uHQQ2T5uFhNFx5AbDBnm6',
'the pulse of organic house': '7bFGHw85oqTjCqQnkbOr0i',
'BAILE: on rotation': '1tHNL2eat97A0b4ZxDcEQp',
'beatport_best_new_melodic': "15IEm4WquzW48cFgtb7Ym7",
# 'anjunadeep_recommends': "1GfH39JcID8aFZ0ZQQVkBk",
# 'progressive_house_and_deep_electronic': "5CMvAWTlDPdZnkleiTHyyo",
'keinemusik': "3U9lafWEnb3uP6JFbI65uq",
'durante_selects': "5dLSmr1SqxNnJDtxvWGHJI",
}

In [7]:
df_list = []

for p_id in pref.values():

    df_tracks = pd.DataFrame()

    curr_len = 0
    offset = 0

    while len(df_tracks) % 100 == 0:
        
        # print(len(df_tracks))
        # print(p_id)

        track_url = f'https://api.spotify.com/v1/playlists/{p_id}/tracks?limit=100&offset={offset}'
        headers = {
            'Authorization': f'Bearer {access_token}'
        }

        r_track = requests.get(track_url, headers=headers)

        if r_track.status_code == 401:

            TOKEN_URL = 'https://accounts.spotify.com/api/token'

            message = config.CLIENT_ID + ':' + config.CLIENT_SECRET
            messageBytes = message.encode('ascii')
            base64Bytes = base64.b64encode(messageBytes)
            base64Message = base64Bytes.decode('ascii')
        
            headers = {
            'Authorization': 'Basic ' + base64Message,
            'Content-Type': 'application/x-www-form-urlencoded'
            }

            pars_refresh = {
                'grant_type': 'refresh_token',
                'refresh_token': refresh_token,
                'redirect_uri': 'http://localhost:8888/callback',
            }

            r_refresh = requests.post(TOKEN_URL, headers=headers, params=pars_refresh)
            access_token = r_refresh.json()['access_token']

            headers = {
            'Authorization': f'Bearer {access_token}'
                    }

            r_track = requests.get(track_url, headers=headers)

        track_ids = [t['track']['uri'] for t in r_track.json()['items']]
        track_names = [t['track']['name'] for t in r_track.json()['items']]
        track_added = [t['added_at'] for t in r_track.json()['items']]

        df_t = pd.DataFrame({
                'name': track_names,
                'id': track_ids,
                'playlist id': p_id

        }, 
        index=pd.to_datetime(track_added)  
        )

        # if df_t['id']
        df_tracks = df_tracks.append(df_t)
        if curr_len == len(df_tracks):
            break
        else:
            curr_len = len(df_tracks)

        offset += 100

    
    df_tracks = df_tracks.sort_index(ascending=False).head(3)
    df_list.append(df_tracks)

df_tracks = pd.concat(df_list)

In [16]:
df_tracks = df_tracks.drop_duplicates()
df_tracks = df_tracks.sample(frac=1)
df_tracks.shape

(15, 3)

In [9]:
three_by_five = '4KgXBrWZqdqVMPrpFNHxpR'
CURR_URL = f'https://api.spotify.com/v1/playlists/{three_by_five}/tracks'
r_current = requests.get(CURR_URL, headers=headers)

In [10]:
delete_uris = [t['track']['uri'] for t in r_current.json()['items']]
# len(delete_uris)

In [11]:
del_uri = []
for i in delete_uris:
    del_uri.append(
        {'uri': i}
    )
del_dict = {'tracks': del_uri}

In [12]:
DELETE_URL = f'https://api.spotify.com/v1/playlists/{three_by_five}/tracks'
r_delete = requests.delete(DELETE_URL, headers=headers, data=json.dumps(del_dict))

In [13]:
track_joins = ','.join(df_tracks['id'].tolist())
ADD_URL = f'https://api.spotify.com/v1/playlists/{three_by_five}/tracks?uris={track_joins}'
r_add = requests.post(ADD_URL, headers=headers)